In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
tf.keras.layers.Flatten(input_shape=(28, 28)),
tf.keras.layers.Dense(128, activation='relu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)


Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2970 - accuracy: 0.9136
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1446 - accuracy: 0.9566
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1070 - accuracy: 0.9682
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0883 - accuracy: 0.9727
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.0718 - accuracy: 0.9779


[0.07183004170656204, 0.9779000282287598]

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(784,)) #2^3*7^2

dense = layers.Dense(64, activation="relu")
x = dense(inputs)

y = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(10)(y)

model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")
#keras.utils.plot_model(model, r"d:\my_first_model.png")
#keras.utils.plot_model(model, r"d:\my_first_model_with_shape_info.png", show_shapes=True)

#Ref: https://keras.io/api/optimizers/
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

model.compile(
loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
optimizer=keras.optimizers.RMSprop(),
metrics=["accuracy"],
)
history = model.fit(x_train, y_train, batch_size=64, epochs=2, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

model.save(r"keras_model.mdl")


Epoch 1/2
750/750 [==============================] - 4s 4ms/step - loss: 0.3380 - accuracy: 0.9031 - val_loss: 0.1937 - val_accuracy: 0.9413
Epoch 2/2
750/750 [==============================] - 5s 6ms/step - loss: 0.1543 - accuracy: 0.9540 - val_loss: 0.1469 - val_accuracy: 0.9575
313/313 - 0s - loss: 0.1396 - accuracy: 0.9581 - 437ms/epoch - 1ms/step
Test loss: 0.13955362141132355
Test accuracy: 0.9581000208854675


Activity 2:

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)


In [ ]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)


In [ ]:
import tensorflow as tf

# Create a model using high-level tf.keras.* APIs
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1]),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=1)
])
model.compile(optimizer='sgd', loss='mean_squared_error') # compile the model
model.fit(x=[-1, 0, 1], y=[-3, -1, 1], epochs=5) # train the model
# (to generate a SavedModel) tf.saved_model.save(model, "saved_model_keras_dir")

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

Epoch 1/5
1/1 [==============================] - 1s 901ms/step - loss: 3.6972
Epoch 2/5
1/1 [==============================] - 0s 18ms/step - loss: 3.5904
Epoch 3/5
1/1 [==============================] - 0s 21ms/step - loss: 3.4777
Epoch 4/5
1/1 [==============================] - 0s 12ms/step - loss: 3.3681
Epoch 5/5
1/1 [==============================] - 0s 10ms/step - loss: 3.2614


In [ ]:
import tensorflow as tf

# Create a model using low-level tf.* APIs
class Squared(tf.Module):
  @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.float32)])
  def __call__(self, x):
    return tf.square(x)
model = Squared()
# (ro run your model) result = Squared(5.0) # This prints "25.0"
# (to generate a SavedModel) tf.saved_model.save(model, "saved_model_tf_dir")
concrete_func = model.__call__.get_concrete_function()

# Convert the model.

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func],
                                                            model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:


# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('keras_model.mdl') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('convt_model.tflite', 'wb') as f:
  f.write(tflite_model)